In [ ]:
import pandas as pd
from scripts.utils import read_gold_data
import scripts.prompts as pr
import scripts.rerank_scripts as rr
from llmusage.hugchat_api_usage import HuggingChat

In [2]:
data_path_name = "data-release" # set data path name

data = read_gold_data(data_path_name)
corpus = data["corpus"]
perspective_queries_train = data["perspective"]["train"]
perspective_queries_dev = data["perspective"]["dev"]
# perspective_queries_test = data["perspective"]["test"]

In [3]:
prediction_path = "" # set path to prediction
predictions = pd.read_json(prediction_path, lines=True)

In [4]:
HUGGINGCHAT_USERNAME = "" # insert valid huggingchat username (I believe the email-adress works as well)
HUGGINGCHAT_PASSWORD = "" # insert the corresponding huggingchat password
answer_with = 'dict'

In [5]:
if answer_with == 'list':
    mixtral_huggingchat = HuggingChat(HUGGINGCHAT_USERNAME, HUGGINGCHAT_PASSWORD, system_prompt="Answer with a python list containing all ranked argument ids")
if answer_with == 'dict':
    mixtral_huggingchat = HuggingChat(HUGGINGCHAT_USERNAME, HUGGINGCHAT_PASSWORD, system_prompt="Answer with a python dictionary containing a score between 0 and 1 for each argument id")

created login


In [6]:
queries = perspective_queries_dev # set correct set of queries

In [ ]:
json_path = "" # this path should lead to an empty json file, where the llm demographic scores per query can be saved right after generating them, in case something crashes and local variables are lost
implicit_demographic_scores = rr.implicit_demographic_scores(predictions, corpus, queries, mixtral_huggingchat, 50, pr.implicit_demographics_prompt, json_path=json_path)

In [ ]:
scores = []
for i in range(len(implicit_demographic_scores)):
    row = list(implicit_demographic_scores[i])
    scores.append({
        'query_id': queries["query_id"].values[i],
        'demographic_scores': row
    })

In [ ]:
implicit_demographic_scores_df = pd.DataFrame(scores)

In [ ]:
alt_json_path = "" # if there was no json path given earlier to save each line of scores individually this is to save the entire table of scores as a json file
implicit_demographic_scores_df.to_json(alt_json_path, orient="records", lines=True)